#Análise de Dados de Diabetes Utilizando Machine Learning

Neste notebook, iremos realizar uma análise de um dataset de diabetes, focando na preparação de dados, análise exploratória
e aplicação de algoritmos de aprendizado de máquina. Usaremos o conjunto de dados `diabetes.csv`, que contém diversas
informações clínicas dos pacientes, como níveis de glicose, pressão arterial e índice de massa corporal (BMI).
Importação de Bibliotecas e Carregamento de Dados

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Carregar o dataset
file_path = 'diabetes.csv'
diabetes_data = pd.read_csv(file_path)

# Exibir as primeiras linhas do dataset
diabetes_data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


#Estatísticas Descritivas
Aqui, apresentamos um resumo estatístico dos dados, que nos permite observar a distribuição das variáveis.


In [ ]:
#Exibir estatísticas descritivas
diabetes_data.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


A análise estatística descritiva do dataset revela os seguintes pontos:


*   Glucose: O valor médio da glicose é 120.9, com valores variando de 0 a 199. Um valor de glicose igual a 0 parece indicar dados ausentes ou inválidos.
*   BloodPressure: A pressão arterial média é 69.1 mm Hg, com alguns valores mínimos de 0, o que não é biologicamente plausível e pode indicar dados ausentes.
*   SkinThickness e Insulin: Ambas as variáveis possuem valores mínimos de 0, o que pode sugerir ausência de medições para alguns pacientes.
*   BMI: O Índice de Massa Corporal (IMC) médio é 31.99, com uma amplitude que vai de 0 a 67.1. Valores de BMI igual a 0 indicam dados faltantes.
Age: A idade média dos pacientes é de 33 anos, com um intervalo variando de 21 a 81 anos.
*   Outcome: A distribuição de pessoas diagnosticadas com diabetes é de aproximadamente 35%, enquanto 65% não têm diabetes.

#Tratamento de Valores Ausentes
Valores zeros em variáveis como glicose, pressão arterial, espessura da pele, insulina e BMI não fazem sentido biológio e clínico. Vamos analisar quais colunas apresentam mais valores 0 e qual seria a sua porcentagem.

In [ ]:
# Substituir valores 0 nas colunas que não podem ter esse valor por suas respectivas medianas
columns_with_zeros = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

# Contar a quantidade de zeros por coluna
zero_counts = (diabetes_data[columns_with_zeros] == 0).sum()

# Exibir o resultado
zero_counts

,0
Glucose,5
BloodPressure,35
SkinThickness,227
Insulin,374
BMI,11


In [ ]:
# Calcular a porcentagem de valores zero na coluna 'Insulin'
total_rows = diabetes_data.shape[0]
zero_insulin_count = (diabetes_data['Insulin'] == 0).sum()
percentage_zero_insulin = (zero_insulin_count / total_rows) * 100

percentage_zero_insulin

48.69791666666667

Quase 50% dos dados contém o valor de insulina igual a 0. Vamos analisar a correlação que os valores de SkinThickness e Insulin possuem com o Outcome, isto é, se a paciente tem ou não Diabetes.

In [ ]:
# Calcular a correlação entre as colunas 'Insulin', 'SkinThickness' e 'Outcome'
correlation_insulin_outcome = diabetes_data['Insulin'].corr(diabetes_data['Outcome'])
correlation_skinthickness_outcome = diabetes_data['SkinThickness'].corr(diabetes_data['Outcome'])

# Exibir os resultados
correlation_insulin_outcome, correlation_skinthickness_outcome

(0.13054795488404794, 0.0747522319183194)

As correlações entre as colunas Insulin e SkinThickness com o Outcome (presença de diabetes) são:

*   Insulin: 0.13
*   SkinThickness: 0.075

Esses valores indicam uma correlação fraca entre essas variáveis e o diagnóstico de diabetes (Outcome). Embora elas possam fornecer informações úteis no modelo, essas variáveis, por si só, não têm uma relação muito forte com o desfecho de diabetes. Sendo assim, vamos seguir fazendo a substituição dos valores 0 pelas medianas de cada coluna.

In [ ]:
# Substituindo 0 pela mediana de cada coluna
for column in columns_with_zeros:
    median_value = diabetes_data[column].median()
    diabetes_data[column] = diabetes_data[column].replace(0, median_value)

# Verificando novamente os dados após a substituição
diabetes_data.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,121.656250,72.386719,27.334635,94.652344,32.450911,0.471876,33.240885,0.348958
std,3.369578,30.438286,12.096642,9.229014,105.547598,6.875366,0.331329,11.760232,0.476951
min,0.000000,44.000000,24.000000,7.000000,14.000000,18.200000,0.078000,21.000000,0.000000
25%,1.000000,99.750000,64.000000,23.000000,30.500000,27.500000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,31.250000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000
